In [1]:
# toydata to test simple algorithm
from datasets import Dataset, DatasetDict, concatenate_datasets
from setretrieval.utils.utils import pickload, pickdump
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from nltk.corpus import wordnet as wn
import random

/accounts/projects/sewonm/prasann/.conda/envs/scaling2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_nouns = [word for synset in wn.all_synsets('n') for word in synset.lemma_names()]
all_nouns = [word for word in all_nouns if "_" not in word]
test_nouns = [{'text': word} for word in all_nouns[:1000]]
# allnouns = Dataset.from_list([{"text": word} for word in all_nouns])
# allnouns.save_to_disk("../propercache/data/datastores/allnouns")

In [3]:
nouns100k = Dataset.from_list([{"text": word} for word in all_nouns[1000:101000]])
nouns10k = Dataset.from_list([{"text": word} for word in all_nouns[1000:11000]])
if False: # do just once to avoid extra I/O
    test_nouns.save_to_disk("../propercache/data/datastores/heldoutnouns")
    nouns100k.save_to_disk("../propercache/data/datastores/nouns100k")
    nouns10k.save_to_disk("../propercache/data/datastores/nouns10k")

In [ ]:
### WE WILL HAVE 2 KINDS OF TASKS
# A: Given a query (one word), return a list of chunks (each chunk has several words)
# B: Given a query (a list of words), return a list of chunks (each chunk is just one noun at a time)

In [ ]:
### NOW STARTING SETUP FOR A

In [ ]:
def generate_train_query_rand(wset, doc_words=10):
    query = random.choice(wset)['text']
    chunkspos = [r['text'] for r in random.choices(wset, k=doc_words)]
    # if query not in chunkspos, randomly swap one of the chunks with query
    if query not in chunkspos:
        swap_idx = random.randint(0, len(chunkspos)-1)
        chunkspos[swap_idx] = query
    chunksneg = [r['text'] for r in random.choices(wset, k=doc_words)]
    return {'query': query, 'positive': " ".join(chunkspos), 'negative': " ".join(chunksneg)}

# TODO this will help us test if minimality somehow makes data better for "hard negatives"
# - Note this is pretty clean minimality, other stuff could / could not matter
def generate_train_query_minimal(wset, doc_words=10):
    query = random.choice(wset)['text']
    chunks = set([r['text'] for r in random.choices(wset, k=doc_words)])
    # TODO sanity check that some weird alphabetization isn't doing something weird
    # make sure query is not in chunks
    if query in chunks:
        chunks.remove(query)
        chunks.add(random.choice(wset)['text'])
    negdata = " ".join(chunks)
    # now randomly swap one of the chunks with query
    swap_idx = random.randint(0, len(chunks)-1)
    chunks = list(chunks)
    chunks[swap_idx] = query
    posdata = " ".join(chunks)
    return {'query': query, 'positive': posdata, 'negative': negdata}

# get 100k train sets for both kinds
dwords = 10
datapoints = 100000
# train100krand = DatasetDict({
#     'train': Dataset.from_list([generate_train_query_rand(nouns10k, dwords) for _ in tqdm(range(datapoints))]),
#     'test': Dataset.from_list([generate_train_query_rand(nouns10k, dwords) for _ in tqdm(range(1000))])
# })
train100kminimal = DatasetDict({
    'train': Dataset.from_list([generate_train_query_minimal(nouns10k, dwords) for _ in tqdm(range(datapoints))]),
    'test': Dataset.from_list([generate_train_query_minimal(nouns10k, dwords) for _ in tqdm(range(1000))])
})
do_save = True
if do_save:
    train100krand.save_to_disk(f"../propercache/data/colbert_training/nountrain{datapoints}rand{dwords}dwords")
    train100kminimal.save_to_disk(f"../propercache/data/colbert_training/nountrain{datapoints}minimal{dwords}dwords")




































































































































































































































100%|██████████| 100000/100000 [00:22<00:00, 4354.07it/s]



100%|██████████| 1000/1000 [00:00<00:00, 4256.82it/s]




Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:00<00:00, 416193.66 examples/s]


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 164038.64 examples/s]




Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:00<00:00, 413210.75 examples/s]


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 101798.55 examples/s]


In [24]:
train100kminimal['train'][0]

{'query': 'snapper',
 'positive': 'Caprimulgidae Mycrosporidia snapper lovebird buffalofish athletics reconnaissance Orectolobus debunking ectoparasite',
 'negative': 'Caprimulgidae Mycrosporidia rescission lovebird buffalofish athletics reconnaissance Orectolobus debunking ectoparasite'}

In [25]:
def singwordquery_eval(evalnouns, trainnouns, dstoresize=100000, tsetsize=500, num_pos=50, docwords=10):
    # make eval test set and datastore
    testqueries = random.choices(evalnouns, k=tsetsize)
    # make eval datastore, with each document have docwords words
    starterdocs = [[r['text'] for r in random.choices(trainnouns, k=docwords)] for _ in tqdm(range(dstoresize))]
    # for each query, randomly choose num_pos docs from starterdocs. For each of these randomly replace one word with query word
    query_poschunks = []
    for query in tqdm(testqueries):
        posinds = random.choices(list(range(dstoresize)), k=num_pos)
        for posind in posinds:
            # randomly replace one word in starterdocs[posind] with query word
            wordind = random.randint(0, docwords-1)
            starterdocs[posind][wordind] = query['text']
        query_poschunks.append(posinds)
    doc_poschunks = Dataset.from_list([{"text": " ".join(d)} for d in starterdocs])
    query_data = Dataset.from_list([
        {
            "question": q, 
            "pos_chunks": [doc_poschunks[p]['text'] for p in query_poschunks[i]], 
            "num_pos_chunks": len(query_poschunks[i])
        } 
        for i, q in enumerate(testqueries)])
    return doc_poschunks, query_data

# get a test set to work with 
tdstore, testset10words50pos = singwordquery_eval(test_nouns, nouns10k, 100000, 500, 50, 10)


















































































































































































































100%|██████████| 100000/100000 [00:21<00:00, 4671.11it/s]






100%|██████████| 500/500 [00:00<00:00, 877.74it/s]


In [26]:
tdstore.save_to_disk("../propercache/data/datastores/evaltdstore10words50pos100k")
testset10words50pos.save_to_disk("../propercache/data/evalsets/testset10words50pos")




Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:00<00:00, 835051.30 examples/s]


Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 17993.12 examples/s]


In [ ]:
# make eval datastore (main thing is making datastore)


In [ ]:
# datapoints = 100000

In [ ]:
### NOW STARTING SETUP FOR B 

In [ ]:
# prepare query, pos, negative data for this stuff
def construct_toy_query(wset, query_words=100, ndps=1, datatype="train"):
    querywords = random.choices(wset, k=query_words)
    querywords = [q['text'] for q in querywords]
    query = " ".join(querywords)
    poslist = random.choices(querywords, k=ndps)
    neglist = []
    for _ in poslist:
        while True: 
            neg = random.choice(wset)['text']
            if neg not in querywords:
                break
        neglist.append(neg)
    if datatype == "train":
        return [{'query': query, 'positive': pos, 'negative': neg} for pos, neg in zip(poslist, neglist)]
    else:
        return {'question': query, 'pos_chunks': querywords, 'numposchunks': len(querywords)}

nouns10k = Dataset.load_from_disk("../propercache/data/datastores/nouns10k")

In [ ]:
def toyquerydset(wset, qwords, ndps, tsize):
    alldata = []
    inddps = tsize // ndps
    for _ in tqdm(range(inddps)):
        alldata.extend(construct_toy_query(wset, qwords, ndps, "train"))
    return Dataset.from_list(alldata)

numwords = 100
tset = 100000
dp_per_query = 100
traindata = DatasetDict({
    'train': toyquerydset(nouns10k, numwords, dp_per_query, tset),
    'test': toyquerydset(nouns10k, numwords, dp_per_query, 1000)
})
traindata.save_to_disk(f"../propercache/data/colbert_training/nountraining{numwords}words{tset}ndps{dp_per_query}")

In [ ]:
# generate eval data following a probability distribution over count of query words
def pdquerydset(wset, qwdistdict, tsize, ndps=1):
    alldata = []
    dcnt = 0
    while dcnt < tsize:
        qw = random.choices(list(qwdistdict.keys()), weights=list(qwdistdict.values()), k=1)[0]
        alldata.extend(construct_toy_query(wset, qw, min(ndps, qw), "train"))
        dcnt += 1
        if dcnt % 10000 == 0:
            print(f"Generated {dcnt} queries")
    return Dataset.from_list(alldata)

# uniform distribution from 5 to 100
qwdistdictuni = {i: 1/96 for i in range(5, 101)}
# power law distribution from 5 to 100
qwdistdictpower = {i: i**(-0.5) for i in range(5, 101)}

# plot power law distribution in qwdistdictpower
x = list(qwdistdictpower.keys())
y = list(qwdistdictpower.values())
plt.plot(x, y)
plt.xlabel("Number of Words in Query")
plt.ylabel("Probability")
plt.title("Power Law Distribution of Query Length")
plt.show()

In [ ]:
# get 100k train set for each distribution
train100kuniform = pdquerydset(nouns10k, qwdistdictuni, 100000)
test100kuniform = pdquerydset(nouns10k, qwdistdictuni, 1000)

train100kpower = pdquerydset(nouns10k, qwdistdictpower, 100000)
test100kpower = pdquerydset(nouns10k, qwdistdictpower, 1000)

DatasetDict({
    'train': train100kuniform,
    'test': test100kuniform
}).save_to_disk("../propercache/data/colbert_training/nountraining100kuniform5_100")

DatasetDict({
    'train': train100kpower,
    'test': test100kpower
}).save_to_disk("../propercache/data/colbert_training/nountraining100kpower5_100")

In [ ]:
# sanity check distributions of 'query' length in train100kuniform, train100kpower
plt.hist([len(q['query'].split()) for q in train100kuniform], alpha=0.5, label='Uniform', range=(5, 100))
plt.hist([len(q['query'].split()) for q in train100kpower], alpha=0.5, label='Power', range=(5, 100))
plt.ylabel("Count") 
plt.xlabel("Number of Words in Query")
plt.title("Distribution of Query Length in Training Sets")
plt.legend()
plt.show()

In [ ]:
evallists = [100] # [1, 5, 10, 25]
for ev in evallists:
    evdata = Dataset.from_list([construct_toy_query(test_nouns, ev, "eval") for _ in tqdm(range(1000))])
    evdata.save_to_disk(f"../propercache/data/evalsets/nountest{ev}/")

In [ ]:
evallists = [1, 5, 10, 25]
testsets = {}
for ev in evallists:
    testsets[ev] = Dataset.load_from_disk(f"../propercache/data/evalsets/nountest{ev}/")

In [ ]:
testsets[1]

In [ ]:
allnouns = Dataset.load_from_disk("../propercache/data/datastores/allnouns")

In [ ]:
# given a test set, convert it to a train set
def test_to_trainset(tset):
    alldata = []
    for row in tset:
        for pos in row['pos_chunks']:
            alldata.append({'query': row['question'], 'positive': pos, 'negative': random.choice(allnouns)['text']})
    return DatasetDict({'train': Dataset.from_list(alldata), 'test': Dataset.from_list(alldata)})

tset = 25
test_to_trainset(testsets[tset]).save_to_disk(f"../propercache/data/colbert_training/nountestset{tset}")

In [ ]:
test_to_trainset(testsets[tset])

In [ ]:
eval10 = Dataset.from_list([construct_toy_query(test_nouns, 10, "eval") for _ in tqdm(range(1000))])
eval100 = Dataset.from_list([construct_toy_query(test_nouns, 100, "eval") for _ in tqdm(range(1000))])

eval10.save_to_disk("../propercache/data/evalsets/nountest10/")
eval100.save_to_disk("../propercache/data/evalsets/nountest100/")

In [ ]:
# TODO need to make eval data

In [ ]:
traindata['train'][4]